# PyZx
## Quantum circuit Simplification, Optimization and Verification 
This amazing open-source aims  to  reduce  the  complexity  and improve  reasoning  with  ZX-diagram. This Python library is avaiable on [Github](github.com/Quantomatic/pyzx) for more detailes check the full [Documentation](https://pyzx.readthedocs.io/en/latest/#)

![PYZX](Picturing_Qprocesses2.png)
[Picturing Quantum Processes
A First Course in Quantum Theory and Diagrammatic Reasoning](https://www.cambridge.org/gb/academic/subjects/physics/quantum-physics-quantum-information-and-quantum-computation/picturing-quantum-processes-first-course-quantum-theory-and-diagrammatic-reasoning?format=HB)

[PyZx Quantomatic](github.com/Quantomatic/pyzx)

### Table of content 

1. [Let's Fly](#Let's_Fly)
     1. [Imports](#Import)
     2. [First Fly in PyZx](#First_Fly)
2. [Let's Go DODO](#GO_DODO)
    1. [Simplification](#Simplification)
    2. [Optimization](#Optimization)
    3. [Verification](#Verification)

## Let's Fly
#### Imports

In [1]:
import pyzx as zx
from pyzx import Circuit
import sys, os; sys.path.append ('..')
import random 
import math 
from fractions import Fraction 
%config InlineBackend.figure_format = 'svg'

#### First_Fly ✈️

First let's start by building a circuit, using qasm notation (Pyzx also supports QC, and Quipper).



<div class="alert alert-info"><h4>Circuit</h4><p>A Circuit consists of a list of Gates, which in turn are small classes containing some information about the gate and how to convert it into various representations, such as ZX-diagrams or the QASM format.

This class allows for importing and exporting circuits to and from PyZX. Providing methods to do gate-level operations, such as converting gates or taking their adjoint.

In [2]:
# start a circuit using qasm 
c = zx.qasm(""" 
qreg q[2];
cx q[0], q[1];
""")
zx.draw(c)

In [3]:
c.to_matrix()

array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j]])

In [6]:
c1 = Circuit(2)
c1.add_gate("CNOT", 0,1)
zx.draw(c1)

## GO_DODO 🦤

Now lets create a new circuit with some more CNOT some S gates and a few rotations 

In [7]:
c = zx.qasm("""
qreg q[2];
rx(0.5*pi) q[1];
t q[0];
cx q[0], q[1];
cx q[1], q[0];
cx q[0], q[1];
tdg q[1];
rx(-0.5*pi) q[0];
""")
zx.draw(c)
print(c.gates)

[XPhase(1,phase=1/2), T(0), CNOT(0,1), CNOT(1,0), CNOT(0,1), T*(1), XPhase(0,phase=-1/2)]


## Simplification
To simplify a circuit using PyZx we need to first convert it to a graph so we can use the built-in diagramatic reasoning. The simplification strategies of PyZX, are built-in hierarchically and each strategy is based on applying some combination of the match and rewrite rules. 


<div class="alert alert-info"><h4>ZX-diagram</h4><p> 
ZX-diagrams are represented in PyZX by instances of the Base Graph class. 
The graphs in PyZX are undirected graphs with typed vertices and edges. 
There are 4 types of vertices:
    
- Boundaries;
- Z-spiders;
- X-spiders;
- H-boxes;
    
Boundary vertices represent an input or an output to the circuit and carry no further information. Z- and X-spider is the usual bread and butter of ZX-diagrams. H-boxes are used in ZH-diagrams as a generalization of the Hadamard gate.

In [8]:
# convert to a zx-diagram 
g = c.to_graph()
g

Graph(14 vertices, 15 edges)

#### Melt these spiders together

In [9]:
zx.simplify.spider_simp(g)
zx.draw(g)

spider_simp: 1. 1.  2 iterations


As we can see above the sipers were fused together using the simple spider simplification. 

If we what to fuse everything together we can try a more greedy reduction stratagy

In [10]:
zx.full_reduce(g, quiet =False)
zx.draw(g)

pivot_simp: 1.  1 iterations
id_simp: 2.  1 iterations
spider_simp: 2.  1 iterations
id_simp: 2.  1 iterations


Actually what we see using the full reduction simplification stratagy is that the previous graph gets reduced to a simple swap gate. As we know from basic quantum computation 3 CNOT gates is a sawp gate, so the reduction is working properlly. 

## Optimization

First of all we need to think what we whant to optimize!

A good way to optimize a circuit is by transform them locally. This envolves reducing the number of gates, since they introduce noise and take time to compute.

Normaly in  **Near-term Quantum Computer** we what to reduce the number of gates explecially the number of 2 qubit gates.
In the future when thinking about a **Full Quantum Computer** the goal is to reduce the number of T gates

<div class="alert alert-info"><h4>ZX-diagram</h4><p> 
The optimization module implements several optimization methods on Circuits.

- basic\_optimization (to run a set of back-and-forth gate commutation and cancellation routines);
- phase\_block\_optimize (does phase polynomial optimization using the TODD algorithm);
- full\_optimize combines these two methods;


In [11]:
# this optimizer uses teleport reduction and full optimization to reduce the number os T Gates 
def t_optimiser(c):
    g = c.to_graph()
    g = zx.simplify.teleport_reduce(g)
    c_opt = zx.Circuit.from_graph(g).split_phase_gates().to_basic_gates()
    return zx.optimize.full_optimize(c_opt).to_basic_gates()

<div class="alert alert-danger"><h4>T Gate</h4><p>
The T gate is a single-qubit operation given by:
$$ T =
\left(\begin{array}{cc} 
1 & 0\\
0 & exp⁡(iπ4)
\end{array}\right)
$$


The T gate is related to the S gate by the relationship  $ S=T^{2}$.

The conjugate transpose of the T gate is defined by:  
$$ T†=
\left(\begin{array}{cc} 
10 & 0\\ 
0 & 5
\end{array}\right)
$$ 


In [13]:
c = zx.Circuit.load('circuits/Fast/mod5_4_before') #implements a clasiccal reverse function
zx.draw(c.to_graph())
print(c.to_basic_gates().stats())

Circuit mod5_4_before on 5 qubits with 63 gates.
        28 is the T-count
        35 Cliffords among which 
        28 2-qubit gates (28 CNOT, 0 other) and
        6 Hadamard gates.


As we se above the nuber of T gates on the circuit is high 

In [15]:
c1 = t_optimiser(c)
zx.draw(c1.to_graph())
print(c1.to_basic_gates().stats())

Circuit  on 5 qubits with 27 gates.
        8 is the T-count
        19 Cliffords among which 
        14 2-qubit gates (12 CNOT, 2 other) and
        2 Hadamard gates.


After the optimization the number of T gates was reduced to 8.

#### But what apends wen we reduce the circuit like this ?

In [18]:
g = c.to_graph()
zx.full_reduce(g, quiet = False)
zx.draw(g)

spider_simp: 10. 7. 4. 4. 3. 1.  6 iterations
id_simp: 6.  1 iterations
spider_simp: 3.  1 iterations
pivot_simp: 4.  1 iterations
pivot_gadget_simp: 12. 4. 3. 1.  4 iterations
id_simp: 8.  1 iterations
spider_simp: 1. 1. 1. 1.  4 iterations
gadget_simp: 4.  1 iterations
lcomp_simp: 4. 4.  2 iterations


##### blue wires represent Hadamard gates 

Even though this graph is a lot compacter than the one we started out with, it no longer looks like a circuit. To fix this we need to be clever and extract a circuit from the ZX-diagram:

In [19]:
c2 = g.copy()
c2.normalize()
c2 = zx.extract_circuit(c2)
zx.draw(c2)

## Verification

#### At the end of the day, what is really important is that the manipulations made to the ZX-diagram preserve its properties and is semanti\cs. 


PyZX offers a straightforward way to do this by allowing ZX-diagrams to be converted into their underlying linear maps using NumPy. To test the equality of diagrams we can then simply test whether all the elements in the two tensors are equal up to a global non-zero scalar. 

Nonetheless, this **uses an exponential amount of memory in terms of qubits**, and this could be a drawback for PyZx use. 

In [20]:
t1 = c.to_graph().to_tensor()
t2 = c1.to_graph().to_tensor()
zx.compare_tensors(t1,t2)

True

But a **clever way to verify** is by using the engine itself to validate equality. 

Considering **c** to be the circuit and **c1** to be the adjoint of c1, then if we combine the two in one circuit **c2** and perform a full simplification we should end up with the identity circuit. 

The function *verify_equality* implements this idea.

In [21]:
c.verify_equality(c1)

True

#### Let's see this in detail 

In [23]:
c2 = c1.copy()
c2.add_circuit(c.adjoint())
g = c2.to_graph()
zx.simplify.full_reduce(g)
zx.draw(g)

As we se above the circuit got reduced to the identity 

In [24]:
# lets add a error to the circuit and see if it works or now 
c1.gates[10] = zx.gates.T(4,adjoint = True)

In [25]:
c.verify_equality(c1)

False

In [26]:
# In detail 
c2 = c1.copy()
c2.add_circuit(c.adjoint())
g = c2.to_graph()
zx.simplify.full_reduce(g)
zx.draw(g)